In [99]:
import os
import pandas as pd
import pydicom
from pydicom import dcmread
from pydicom.fileset import FileSet
import os, datetime

def read_folders(path):
    """
    Reads all folders in the given path.
    Args:
        path (str): The path to the directory.
    Returns:
        list: A list of folder names in the directory.
    """
    folderName = []
    fileName = []
    seriesNum = []
    try:
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            #print(item_path)
            if os.path.isdir(item_path):
                firstfile = os.listdir(item_path)[0]
                file_path = os.path.join(item_path, firstfile)
                #print(file_path)
                if os.path.isfile(file_path):
                    header_data = dcmread(file_path)
                    pid = header_data.PatientID
                    mod = header_data.SeriesDescription  
                    series = header_data.SeriesNumber
                    folderName.append(item)
                    fileName.append(mod)
                    seriesNum.append(series)

    except FileNotFoundError:
        print(f"Error: Directory not found: {path}")
        return None
    except PermissionError:
        print(f"Error: Permission denied: {path}")
        return None
    return folderName, seriesNum, fileName, pid

In [ ]:
destination = 
dicomFolder = 

fname, series, mod, pid = read_folders(dicomFolder)

df = pd.DataFrame([fname, series, mod]).T
df.columns = ['Folder', 'SereisNumber', 'SeriesDescription']
dataFile = os.path.join(destination,pid + '_modalities.csv')
df.to_csv(dataFile)

In [ ]:
import SimpleITK as sitk
import os, glob, subprocess
import dicom2nifti
import pandas as pd
import pydicom
from pydicom import dcmread
from pydicom.fileset import FileSet

def read_folders(path):
    """
    Reads all folders in the given path.
    Args:
        path (str): The path to the directory.
    Returns:
        list: A list of folder names in the directory.
    """
    folderName = []
    fileName = []
    seriesNum = []
    try:
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            #print(item_path)
            if os.path.isdir(item_path):
                firstfile = os.listdir(item_path)[0]
                file_path = os.path.join(item_path, firstfile)
                #print(file_path)
                if os.path.isfile(file_path):
                    header_data = dcmread(file_path)
                    pid = header_data.PatientID
                    mod = header_data.SeriesDescription  
                    series = header_data.SeriesNumber
                    folderName.append(item)
                    fileName.append(mod)
                    seriesNum.append(series)

    except FileNotFoundError:
        print(f"Error: Directory not found: {path}")
        return None
    except PermissionError:
        print(f"Error: Permission denied: {path}")
        return None
    return folderName, seriesNum, fileName, pid

def convert_dcm_to_nii(dcm_dir, nii_out_path):
    """Converts DICOM files in a directory to a NIfTI file."""
    dicom2nifti.convert_directory(dcm_dir, nii_out_path, compression=True, reorient=True)

In [100]:
dicomFolder = 
destination = 

fname, series, mod, pid = read_folders(dicomFolder)

df = pd.DataFrame([fname, series, mod]).T
df.columns = ['Folder', 'SereisNumber', 'SeriesDescription']
dataFile = os.path.join(destination,pid + '_modalities.csv')
df.to_csv(dataFile)
data = pd.read_csv(dataFile))

images = {'dti': 'DTI Tractography', 'avgdc': 'AvDC (10^-6 mm²/s): b1000', 'dsc': 'Ax DSC-GRE', 'dce': 'Ax DCE', 'ktrans': 'IB Extended Tofts Ktrans (linear)', 'cbv': 'IB rCBV (leakage-corrected)'}

for mod, image in images.items():
    matched_idx = dataFile[dataFile['SeriesDescription'] == image].index[0]
    value = dataFile.loc[matched_idx, 'Folder']

    path2dcm = os.path.join(dicomFolder, value)   
    os.makedirs(os.path.join(destination, 'conv/nifti', mod), exist_ok = True)
    path2nii = os.path.join(destination, 'conv/nifti', mod)
    
    convert_dcm_to_nii(path2dcm, path2nii) 
    
    path2newnii = glob.glob(os.path.join(destination, 'conv/nifti', mod, '*.nii.gz'))

    # Load the image
    image = sitk.ReadImage(path2newnii)

    # Get the size of the image
    size = image.GetSize()

    # Iterate over the 4th dimension (time)
    for t in range(size[3]):
        # Extract the 3D volume at time t
        volume = image[:, :, :, t]

        # Do something with the volume, e.g., save it
        dti_out = os.path.join(destination, 'conv/nifti', mod, f'{mod}_{t}.nii.gz')
        sitk.WriteImage(volume, dti_out)

    os.makedirs(os.path.join(destination, 'conv/ss'), exist_ok = True)

    img = os.path.join(destination, 'conv/nifti', mod, mod + '_0.nii.gz')
    img_ss = os.path.join(destination, 'conv/ss', mod + '_ss.nii.gz')

    command1 = ['hd-bet', '-i',  img, '-o', img_ss, '--save_bet_mask']
    c1 = subprocess.call(command1)


/rsrch1/ip/rmuthusivarajan/conda_env/lib/python3.11/site-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = 


########################
If you are using hd-bet, please cite the following papers:

Isensee F, Schell M, Tursunova I, Brugnara G, Bonekamp D, Neuberger U, Wick A, Schlemmer HP, Heiland S, Wick W, Bendszus M, Maier-Hein KH, Kickingereder P. Automated brain extraction of multi-sequence MRI using artificial neural networks. arXiv preprint arXiv:1901.11341, 2019.

Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
########################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting dsc_ss_bet.nii.gz:
perform_everything_on_device: True


100%|██████████| 8/8 [00:06<00:00,  1.23it/s]


sending off prediction to background worker for resampling and export
done with dsc_ss_bet.nii.gz


/rsrch1/ip/rmuthusivarajan/conda_env/lib/python3.11/site-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = 

...

sending off prediction to background worker for resampling and export
done with ktrans_ss_bet.nii.gz


/rsrch1/ip/rmuthusivarajan/conda_env/lib/python3.11/site-packages/nnunetv2/inference/predict_from_raw_data.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = 


########################
If you are using hd-bet, please cite the following papers:

Isensee F, Schell M, Tursunova I, Brugnara G, Bonekamp D, Neuberger U, Wick A, Schlemmer HP, Heiland S, Wick W, Bendszus M, Maier-Hein KH, Kickingereder P. Automated brain extraction of multi-sequence MRI using artificial neural networks. arXiv preprint arXiv:1901.11341, 2019.

Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
########################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting cbv_ss_bet.nii.gz:
perform_everything_on_device: True


100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


sending off prediction to background worker for resampling and export
done with cbv_ss_bet.nii.gz
